In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox, Layout, Box
from IPython.display import clear_output
import _pickle as pickle
from w2v_model import word2vec
import get_answer

In [2]:
ob = word2vec()
path = "../data_prep/data/"
res = get_answer.get_dir_list(path)
print (res)
get_answer.ob = ob
driver = get_answer.Driver()
fp = open(path+"PR.txt", 'w')
for cat in res:
    print ("\n\n\n\n==================================\n\n\n\n")
    print ("                 "+cat+"               ")
    print ("\n\n\n\n==================================\n\n\n\n")
    fp.write(cat)
    fp.write("\n")
    driver.run_w2v(path+cat, fp)
fp.close()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/shashwat/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /home/shashwat/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /home/shashwat/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/shashwat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['Musical_Instruments', 'PR.txt']








                 Musical_Instruments               








Reading files ../data_prep/data/Musical_Instruments/Musical_Instruments_Review.json


2018-04-22 15:53:00,464 : INFO : collecting all words and their counts
2018-04-22 15:53:00,465 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-22 15:53:00,475 : INFO : collected 25886 word types from a corpus of 25886 raw words and 1 sentences
2018-04-22 15:53:00,477 : INFO : Loading a fresh vocabulary
2018-04-22 15:53:00,571 : INFO : min_count=1 retains 25886 unique words (100% of original 25886, drops 0)
2018-04-22 15:53:00,572 : INFO : min_count=1 leaves 25886 word corpus (100% of original 25886, drops 0)
2018-04-22 15:53:00,636 : INFO : deleting the raw counts dictionary of 25886 items
2018-04-22 15:53:00,637 : INFO : sample=0.001 downsamples 0 most-common words
2018-04-22 15:53:00,638 : INFO : downsampling leaves estimated 25886 word corpus (100.0% of prior 25886)


Performing Word2Vec


2018-04-22 15:53:00,695 : INFO : estimated required memory for 25886 words and 100 dimensions: 33651800 bytes
2018-04-22 15:53:00,695 : INFO : resetting layer weights
2018-04-22 15:53:00,962 : INFO : training model with 3 workers on 25886 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2018-04-22 15:53:00,971 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-22 15:53:00,972 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-22 15:53:01,001 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-22 15:53:01,002 : INFO : EPOCH - 1 : training on 25886 raw words (10000 effective words) took 0.0s, 267084 effective words/s
2018-04-22 15:53:01,007 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-22 15:53:01,008 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-22 15:53:01,031 : INFO : worker thread finished; awaiting finish of 0 more threads
2018

Dumping Pickles
Total products with Questions = 380
Processing
Category Precision : 87.46052631578947

Category Recall : 59.33121063800627

#######################









                 PR.txt               








Reading files ../data_prep/data/PR.txt/PR.txt_Review.json


NotADirectoryError: [Errno 20] Not a directory: '../data_prep/data/PR.txt/PR.txt_Review.json'

In [3]:
def ask_question(b):
    print ("in")
    global category, product, question_text    
    get_answer.review_2_sent(question_text.value, 4, product.value)

In [4]:
categories = ["Select...", "Musical Instruments"]
# "Arts Crafts and Sewing", "Automotive", "Baby", "Beauty", \
# "Cell Phones and Accessories", "Clothing Shoes and Jewelry", "Electronics", \
# "Grocery and Gourmet Food", "Health and Personal Care", "Home and Kitchen", \
# "Industrial and Scientific", , "Office Products", \
# "Patio Lawn and Garden", "Pet Supplies", "Software", "Sports and Outdoors"
products = ["P1", "B00005ML71", "B000VTPR08", "P4", "P5"]
# with open("./cs670project/data_prep/data/Musical_Instruments/Musical_Instruments_QA.json", 'rb') as f:
#     data = f.read()
#     print (type(f))
#     data = pickle.load(data[1:-1])

# print (data[1:-1])

In [9]:
category = widgets.Dropdown(
    options=categories,
    value='Select...',
    description='Category:',
    disabled=False,
)

product = widgets.Dropdown(
    options=products,
    value='P1',
    description='Product:',
    disabled=False,
)

question_text = widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='Question:',
    disabled=False
)

submit = widgets.Button(
    description='Ask me!',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)
submit.on_click(ask_question)

box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    border='none',
                    width='60%')

items = [category, product, question_text, submit]
box =Box(children=items, layout=box_layout)

In [10]:
box

Box(children=(Dropdown(description='Category:', options=('Select...', 'Musical Instruments'), value='Select...…

in
Answer1:  Look no further these are great bags.Very pleased. 

Answer2:  The Fender's straps are great.The Finder's padding is a bit better than my other bag also, but, of course, neither is as good as hard case.The guitar I tote in this bag is a Martin HD-28.But, it can also my Blueridge BR-180 or my Seagull S6 Coastal Spruce.
